# Chat Completions Part 4

Make sure you have an environment variable called OPENAI_API_KEY set with your API key.

## logprobs (boolean or null)
Whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message.

Here's the revised text without the highlighting around the word "logprobs":

---

## Overview
Log probabilities are instrumental in enhancing the functionality and reliability of large language models across a variety of scenarios. Below is a detailed explanation of their use cases:

### Classification Tasks
Log probabilities are extremely valuable in classification tasks like categorizing news headlines or identifying sentiments within texts. They provide the log probability for each potential category, allowing developers to assess the model's confidence in its classifications. This information can be leveraged to set thresholds for accepting predictions or deciding when further review is necessary, thus enhancing both the reliability and accuracy of automated systems.

### Retrieval (Q&A) Evaluation
In question-answering systems, log probabilities measure the confidence level of the answers provided. This helps determine the relevance and correctness of the answers, minimizing errors and preventing the model from presenting incorrect information when the context isn't sufficiently clear.

### Autocomplete
In autocomplete systems, such as those used in search engines or text editors, log probabilities enable the system to suggest the most likely completions by analyzing the log probabilities of different continuations as a user types, thus enhancing the user experience by making typing faster and more precise.

### Token Highlighting and Outputting Bytes
Log probabilities aid in token highlighting by showing which tokens the model considers most probable, thereby helping in text analysis or debugging of model outputs. The bytes parameter is crucial for accurately reproducing texts with special characters or emojis, ensuring correct display across different platforms.

### Calculating Perplexity
Perplexity, calculated using the log probabilities of the tokens generated by the model, is a measure of the model's certainty about its output. Lower perplexity indicates higher confidence. This metric is vital for comparing model performances across various tasks or configurations.

### Moderation
Log probabilities support moderation efforts by identifying high-probability words or phrases that match known patterns of inappropriate content, enabling automated filtering of harmful language.

### Keyword Selection
In content generation and SEO, log probabilities assist in selecting keywords that are most likely to enhance the visibility of web content, allowing marketers to optimize their content more effectively.

### Improving Prompts and Interpretability of Outputs
Log probabilities provide insights into how different prompts affect the likelihood of various responses, which is crucial for refining interactions and designing more effective prompts.

### Token Healing
In text correction or "token healing," log probabilities help identify and replace unlikely tokens or phrases with more probable alternatives, thus improving the grammatical correctness or coherence of the text.


In [12]:
# normal chat example without logprob data
from openai import OpenAI
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[ 
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Use two words to describe the universe."}
    ],
    temperature=1,
    max_tokens=256,
    stop=None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

print(completion.choices[0].message.content)

Expansive mystery.


In [40]:
# chat example with extracting one set of logprob data
from openai import OpenAI
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[ 
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Use two words to describe the universe."}
    ],
    temperature=1,
    max_tokens=256,
    stop=None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    logprobs=True,
)

print(completion.choices[0].message.content)
print("\n\n")
print(completion.choices[0].logprobs)
print("\n\n")
print(completion.choices[0].logprobs.content[0].token)
print(completion.choices[0].logprobs.content[0].logprob)

# calculating the probability of the token
probability = np.round(np.exp(completion.choices[0].logprobs.content[0].logprob)*100,2)
message = f"{probability}%" if probability < 0.01 else f"{probability:.2f}%"
print(message)
print("\n\n")



Expansive mystery.



ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Exp', bytes=[69, 120, 112], logprob=-2.120557, top_logprobs=[]), ChatCompletionTokenLogprob(token='ans', bytes=[97, 110, 115], logprob=-0.000444374, top_logprobs=[]), ChatCompletionTokenLogprob(token='ive', bytes=[105, 118, 101], logprob=-1.18755715e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token=' mystery', bytes=[32, 109, 121, 115, 116, 101, 114, 121], logprob=-0.2989575, top_logprobs=[]), ChatCompletionTokenLogprob(token='.', bytes=[46], logprob=-0.15134661, top_logprobs=[])])



Exp
-2.120557
12.00%





In [44]:
import numpy as np
from openai import OpenAI

# Initialize the client
client = OpenAI()

# Create a chat completion request
completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[ 
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Use two words to describe the universe."}
    ],
    temperature=1,
    max_tokens=256,
    stop=None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    logprobs=True,
)

# Retrieve the response
response = completion.choices[0].message.content
logprobs = completion.choices[0].logprobs

print("Response:", response)
print("\nToken probabilities:\n")

# Iterate through each token logprob entry
for current_logprob in logprobs.content:
    token = current_logprob.token
    logprob = current_logprob.logprob
    probability = np.round(np.exp(logprob) * 100, 2)
    message = f"{probability}%" if probability < 0.01 else f"{probability:.2f}%"
    print(f"Token: {token}, Logprob: {logprob}, Probability: {message}")


Response: Infinitely vast.

Token probabilities:

Token: Inf, Logprob: -2.0300112, Probability: 13.13%
Token: initely, Logprob: -1.3856493e-06, Probability: 100.00%
Token:  vast, Logprob: -0.7190319, Probability: 48.72%
Token: ., Logprob: -0.9741263, Probability: 37.75%


## top_logprobs 
An integer between 0 and 20 specifying the number of most likely tokens to return at each token position, each with an associated log probability. logprobs must be set to true if this parameter is used.

In [64]:
# chat example with extracting one set of logprob data
from openai import OpenAI
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[ 
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Use two words to describe the universe."}
    ],
    temperature=1,
    max_tokens=256,
    stop=None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    logprobs=True,
    top_logprobs=2,
)

print(completion.choices[0].message.content)
print("\n\n")
print(completion.choices[0].logprobs)
print("\n\n")
print(completion.choices[0].logprobs.content[0].top_logprobs[0].token)
print(completion.choices[0].logprobs.content[0].top_logprobs[0].logprob)
# calculating the probability of the token
probability = np.round(np.exp(completion.choices[0].logprobs.content[0].top_logprobs[0].logprob)*100,2)
message = f"{probability}%" if probability < 0.01 else f"{probability:.2f}%"
print(message)
print("\n")

print(completion.choices[0].logprobs.content[0].top_logprobs[1].token)
print(completion.choices[0].logprobs.content[0].top_logprobs[1].logprob)
# calculating the probability of the token
probability = np.round(np.exp(completion.choices[0].logprobs.content[0].top_logprobs[1].logprob)*100,2)
message = f"{probability}%" if probability < 0.01 else f"{probability:.2f}%"
print(message)
print("\n")



Vast, mysterious



ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='V', bytes=[86], logprob=-0.45684347, top_logprobs=[TopLogprob(token='V', bytes=[86], logprob=-0.45684347), TopLogprob(token='Inf', bytes=[73, 110, 102], logprob=-1.6443435)]), ChatCompletionTokenLogprob(token='ast', bytes=[97, 115, 116], logprob=0.0, top_logprobs=[TopLogprob(token='ast', bytes=[97, 115, 116], logprob=0.0), TopLogprob(token='as', bytes=[97, 115], logprob=-18.3125)]), ChatCompletionTokenLogprob(token=',', bytes=[44], logprob=-1.3542051, top_logprobs=[TopLogprob(token=' mystery', bytes=[32, 109, 121, 115, 116, 101, 114, 121], logprob=-0.41670513), TopLogprob(token=',', bytes=[44], logprob=-1.3542051)]), ChatCompletionTokenLogprob(token=' mysterious', bytes=[32, 109, 121, 115, 116, 101, 114, 105, 111, 117, 115], logprob=-0.014296548, top_logprobs=[TopLogprob(token=' mysterious', bytes=[32, 109, 121, 115, 116, 101, 114, 105, 111, 117, 115], logprob=-0.014296548), TopLogprob(token=' M', bytes=[32, 

In [68]:
from openai import OpenAI
import numpy as np

# Initialize the OpenAI client
client = OpenAI()

# Create a chat completion request
completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[ 
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Use two words to describe the universe."}
    ],
    temperature=1,
    max_tokens=256,
    stop=None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    logprobs=True,
    top_logprobs=5,  # Change this to fetch all top logprobs
)

# Print the resulting message
print(completion.choices[0].message.content)
print("\n\n")

# Loop through each token logprob and print all the top logprobs for each token
for token_logprob in completion.choices[0].logprobs.content:
    print(f"Token: {token_logprob.token}")
    print("Top Logprobs:")
    for top_logprob in token_logprob.top_logprobs:
        print(f"  {top_logprob.token}: {top_logprob.logprob}")
        probability = np.round(np.exp(top_logprob.logprob)*100,2)
        print(f"  Probability: {probability}%")
    print("\n")


Vast mystery.



Token: V
Top Logprobs:
  V: -0.45684347
  Probability: 63.33%
  Inf: -1.6443435
  Probability: 19.31%
  Exp: -2.3005934
  Probability: 10.02%
  In: -2.7068434
  Probability: 6.67%
  End: -6.1443434
  Probability: 0.21%


Token: ast
Top Logprobs:
  ast: 0.0
  Probability: 100.0%
  as: -18.328125
  Probability: 0.0%
  AST: -18.765625
  Probability: 0.0%
   ast: -21.3125
  Probability: 0.0%
  aste: -22.609375
  Probability: 0.0%


Token:  mystery
Top Logprobs:
   mystery: -0.4577507
  Probability: 63.27%
  ,: -1.2702507
  Probability: 28.08%
   Mystery: -3.1765008
  Probability: 4.17%
  ly: -4.020251
  Probability: 1.79%
  .: -4.364001
  Probability: 1.27%


Token: .
Top Logprobs:
  <|end|>: -0.49822634
  Probability: 60.76%
  .: -0.93572634
  Probability: 39.23%
  <|end|>: -9.716976
  Probability: 0.01%
   : -10.451351
  Probability: 0.0%
  .
: -13.420101
  Probability: 0.0%




## Classification Confidence with Logprobs
Let's say we want to create a system to classify news articles into a set of pre-defined categories. Without logprobs, we can use Chat Completions to do this, but it is much more difficult to assess the certainty with which the model made its classifications.

Now, with logprobs enabled, we can see exactly how confident the model is in its predictions, which is crucial for creating an accurate and trustworthy classifier. For example, if the log probability for the chosen category is high, this suggests the model is quite confident in its classification. If it's low, this suggests the model is less confident. This can be particularly useful in cases where the model's classification is not what you expected, or when the model's output needs to be reviewed or validated by a human.

In [81]:
# set up our variables
CLASSIFICATION_PROMPT = """You will be given a headline of a news article.
Classify the article into one of the following categories: Technology, Politics, Sports, and Art.
Return only the name of the category, and nothing else.
MAKE SURE your output is one of the four categories stated."""

headlines = [
    "Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.",
    "Local Mayor Launches Initiative to Enhance Urban Public Transport.",
    "Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut",
    "Local Artist Wins Prestigious Award for Creating a Robot to Paint Abstract Artwork.",
    "Startup Develops New App to Help Grassroots Campaigns Increase Voter Engagement",
    "Former Football Star Elected Mayor Promises to Revitalize City's Public Spaces with Innovative Designs",
    "Tech Conference Features Interactive Exhibit on the Evolution of Digital Art",
    "University Research Team Uses AI to Analyze Historical Election Strategies",
    "Documentary on the Intersection of Sports and Art Premieres at International Film Festival",
    "Politician’s Controversial Tweet About National Team Selection Sparks Debate",
    "Emerging Technologies Take Center Stage at Annual Sports Science Symposium",
    "Biotech Firm Collaborates with Art Institute to Explore Genetic Bases of Creativity",
    "Famous Artist and Soccer Star Collaborate on a Series of Political Graffiti Murals",
    "Silicon Valley and Hollywood Unite to Develop a Virtual Reality Series About Ancient Olympic Games",
    "Government Funds New Technology for Online Platforms to Boost Civic Engagement in Local Communities",
    "Major Tech Company Accused of Influencing Political Elections Through Social Media Algorithms",
    "Renowned Sculptor Uses Recycled Electronics to Create Art Commenting on E-waste",
    "International Summit Discusses the Role of Sports in Promoting Global Peace and Politics",
    "Activists Use Sports Events as Platforms for Artistic Performances Highlighting Social Issues",
    "New Study Shows How Digital Platforms Can Enhance Art Education in Schools"
]



In [79]:
# normal loop through the headlines with categorization
for headline in headlines:
    print(f"\nHeadline: {headline}")
    
    completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[ 
        {"role": "system", "content": CLASSIFICATION_PROMPT},
        {"role": "user", "content": f"Headline: {headline}"},
    ],
    temperature=1,
    max_tokens=256,
    stop=None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    logprobs=True,
    )
    
    print(f"Category: {completion.choices[0].message.content}\n")


Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.
Category: Technology


Headline: Local Mayor Launches Initiative to Enhance Urban Public Transport.
Category: Politics


Headline: Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut
Category: Art


Headline: Local Artist Wins Prestigious Award for Creating a Robot to Paint Abstract Artwork.
Category: Art


Headline: Startup Develops New App to Help Grassroots Campaigns Increase Voter Engagement
Category: Technology


Headline: Former Football Star Elected Mayor Promises to Revitalize City's Public Spaces with Innovative Designs
Category: Politics


Headline: Tech Conference Features Interactive Exhibit on the Evolution of Digital Art
Category: Art


Headline: University Research Team Uses AI to Analyze Historical Election Strategies
Category: Technology


Headline: Documentary on the Intersection of Sports and Art Premieres at International Film Festival
Category: Art


Headlin

In [85]:

for headline in headlines:
    print(f"\nHeadline: {headline}")
    
    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[ 
            {"role": "system", "content": CLASSIFICATION_PROMPT},
            {"role": "user", "content": headline},
        ],
        temperature=1,
        max_tokens=256,
        stop=None,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        logprobs=True
    )
    
    response_content = completion.choices[0].message.content
    first_token = completion.choices[0].logprobs.content[0].token
    first_token_logprob = completion.choices[0].logprobs.content[0].logprob
    
    print(response_content)
    print(f"First token: {first_token}")
    print(f"Logprob of first token: {first_token_logprob}")

    # Calculating the probability of the token
    probability = np.exp(first_token_logprob) * 100
    formatted_probability = f"{probability:.2f}%" if probability < 0.01 else f"{probability:.2f}%"
    print(f"Probability of first token: {formatted_probability}")



Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.
Technology
First token: Technology
Logprob of first token: -1.9361265e-07
Probability of first token: 100.00%

Headline: Local Mayor Launches Initiative to Enhance Urban Public Transport.
Politics
First token: Politics
Logprob of first token: -4.365741e-06
Probability of first token: 100.00%

Headline: Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut
Art
First token: Art
Logprob of first token: -1.3856493e-06
Probability of first token: 100.00%

Headline: Local Artist Wins Prestigious Award for Creating a Robot to Paint Abstract Artwork.
Art
First token: Art
Logprob of first token: -0.012509655
Probability of first token: 98.76%

Headline: Startup Develops New App to Help Grassroots Campaigns Increase Voter Engagement
Technology
First token: Technology
Logprob of first token: -1.0537016
Probability of first token: 34.86%

Headline: Former Football Star Elected Mayor Promises 